In [2]:
# Using InstaLoader to retrieve the number of posts made by each political party from the 6th of januray to the 9th of March
# creating a csv file with the columns 'Political Party', 'Username' and 'Post Date'

import instaloader
from datetime import datetime, timedelta
import csv

L = instaloader.Instaloader()

# The usernames of the political parties on Instagram
party_usernames = ["partido_socialista", "partidosocialdemocrata", "partidochega", "liberalpt", "blocodeesquerdaoficial", "pcp.pt", "partidolivre", "partido_pan"]

# Define the start and end date of the time period
start_date = datetime(2024, 1, 6)
end_date = datetime(2024, 3, 9) 

# Save it into a CSV file
with open('frequency_of_the_instagram_posts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Political Party', 'Username', 'Post Date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    # Iterate through each party and retrived their posts made in the time period
    for party in party_usernames:
        print(f"Fetching posts for {party}")
        try:
            profile = instaloader.Profile.from_username(L.context, party)

            # Download those posts
            posts = profile.get_posts()
            for post in posts:
                if start_date <= post.date_utc <= end_date:
                    writer.writerow({'Political Party': party, 'Username': profile.username, 'Post Date': post.date_utc.strftime("%Y-%m-%d %H:%M:%S")})
                elif post.date_utc < start_date:
                    break  # Stop fetching posts if the post date is before the start date
        except instaloader.exceptions.InstaloaderException as e:
            print(f"Error fetching posts for {party}: {e}")  # this is to see if there were any errors getting the data 
        except Exception as e:                                        # for some party
            print(f"An unexpected error occurred for {party}: {e}")

print("Data saved to frequency_of_the_instagram_posts.csv")

Fetching posts for partido_socialista
Fetching posts for partidosocialdemocrata
Fetching posts for partidochega
Fetching posts for liberalpt
Fetching posts for blocodeesquerdaoficial
Fetching posts for pcp.pt
Fetching posts for partidolivre
Fetching posts for partido_pan
Data saved to frequency_of_the_instagram_posts.csv


In [5]:
# Since the names on the 'Political Party' column were the Instagram usernames, this is to replace that with the
# parties names

import pandas as pd

df = pd.read_csv('frequency_of_the_instagram_posts.csv')

# Define the names for each party
party_mapping = {
    'partido_socialista': 'PS',
    'partidosocialdemocrata': 'PPD/PSD.CDS-PP.PPM',
    'partidochega': 'CH',
    'liberalpt': 'IL',
    'blocodeesquerdaoficial': 'B.E.',
    'pcp.pt': 'PCP',
    'partidolivre': 'L',
    'partido_pan': 'PAN'
}

# Replace old party names with new ones
df['Political Party'] = df['Political Party'].replace(party_mapping)

# Update the file
df.to_csv('frequency_of_the_instagram_posts.csv', index=False)